In [1]:
import pandas as pd

RECORD_DATA_FP = 'data/tmp/6.24号以后reversible_lanes_data.xls'

record_df = pd.read_excel(RECORD_DATA_FP)
print(record_df.loc[0, 'json_data'])

{"duration": 1800, "laneData": [{"flow": 356, "laneId": 17, "queueNum": 10, "queueLength": 56}, {"flow": 298, "laneId": 18, "queueNum": 10, "queueLength": 60}], "timestamp": 1685624408}


In [2]:
import json

LANE_IDS = {16, 17, 18, 19, 30, 31, 32, 33}

def lane_flow_col_ids():
    return  [f'lane{lane_id}Flow' for lane_id in LANE_IDS]

def _filter_road_status(info: dict):
    if not 'laneData' in info:
        return False

    all_lanes = {item['laneId'] for item in info['laneData']}
    if not LANE_IDS == all_lanes:
        return False
    return True

def _unpack_data(series: pd.Series):
    info =  series['json_data']
    new_series = pd.Series({'timestamp': info['timestamp'], 'duration': info['duration']})
    for lane_data in info['laneData']:
        lane_id = lane_data['laneId']
        flow = lane_data['flow']
        new_series[f'lane{lane_id}Flow'] = flow
    return new_series

    

record_df['json_data'] = record_df['json_data'].apply(json.loads)
traffic_state_data_index = record_df['json_data'].map(_filter_road_status)
related_df = record_df.loc[traffic_state_data_index, :]

tf_df = related_df.apply(_unpack_data, axis=1)
print(tf_df)

       timestamp  duration  lane16Flow  lane17Flow  lane18Flow  lane19Flow  \
6     1686047540      1200         282         369          54         477   
7     1686048740      1200         204         336          30         531   
8     1686049940      1200         234         300          21         399   
10    1686051140      1200         132         279          54         405   
11    1686052340      1200         144         276          30         378   
...          ...       ...         ...         ...         ...         ...   
1557  1687852801      1200         105         237         126         327   
1558  1687854001      1200         156         255         168         297   
1559  1687855201      1200         216         339         138         381   
1561  1687856400      1200         255         405          57         432   
1562  1687857601      1200         213         336          78         504   

      lane30Flow  lane31Flow  lane32Flow  lane33Flow  
6       

In [3]:
## accumulate data
import time
from collections import defaultdict
UPDATE_INTERVAL = 20  # 更新间隔时间

accumulate_weekday_flow = defaultdict(dict)
accumulate_weekend_flow = defaultdict(dict)

def calculate_avg_flow(flow_data):
    for period_decimal, _flow in flow_data.items():
        for lane_id in _flow.keys():
            all_flow = _flow[lane_id]
            _flow[lane_id] = sum(all_flow) / len(all_flow)
    return dict(sorted(flow_data.items(), key=lambda x: x[0]))

for index, row in tf_df.iterrows():
    time_struct = time.localtime(row['timestamp'])
    time_hour_decimal = (time_struct.tm_hour * 60 + time_struct.tm_min) // UPDATE_INTERVAL    
    data_container = accumulate_weekend_flow if time_struct.tm_wday in (5, 6) else accumulate_weekday_flow
    for lane_id, lane_col_name in zip(LANE_IDS, lane_flow_col_ids()):
        lane_period_data = data_container[time_hour_decimal].setdefault(lane_id, [])
        lane_period_data.append(row[lane_col_name])

weekday_flow = calculate_avg_flow(accumulate_weekday_flow)
weekend_flow = calculate_avg_flow(accumulate_weekend_flow)

In [4]:
from lib.SPAT import Movement, Direction, Turn
from lib.state import TurnDemand, DayLanePlan, PlanDuration
from src.lane_change import StaticIntersectionController, VarianceLane, LaneAllocation, VMS

STRAIGHT_SAT_RATE = 1600
TURN_SAT_RATE = 900
LEFT_SAT_RATE = 1300
RIGHT_SAT_RATE = 1400

LANE_MOVEMENT_MAPPING = {
    16: Movement(Direction.EAST, Turn.STRAIGHT),
    17: Movement(Direction.EAST, Turn.STRAIGHT),
    18: Movement(Direction.EAST, Turn.STRAIGHT),
    19: Movement(Direction.EAST, Turn.RIGHT_TURN),
    30: Movement(Direction.WEST, Turn.LEFT),
    31: Movement(Direction.WEST, Turn.STRAIGHT),
    32: Movement(Direction.WEST, Turn.STRAIGHT),
    33: Movement(Direction.WEST, Turn.STRAIGHT)
}

# 只有东进口变
PEAK_LANE_MOVEMENT_MAPPING1 = {
    16: Movement(Direction.EAST, Turn.STRAIGHT),
    17: Movement(Direction.EAST, Turn.STRAIGHT),
    18: Movement(Direction.EAST, Turn.TURN),
    19: Movement(Direction.EAST, Turn.RIGHT),
    30: Movement(Direction.WEST, Turn.LEFT),
    31: Movement(Direction.WEST, Turn.STRAIGHT),
    32: Movement(Direction.WEST, Turn.STRAIGHT),
    33: Movement(Direction.WEST, Turn.STRAIGHT)
}

# 两者都变
PEAK_LANE_MOVEMENT_MAPPING2 = {
    16: Movement(Direction.EAST, Turn.STRAIGHT),
    17: Movement(Direction.EAST, Turn.STRAIGHT),
    18: Movement(Direction.EAST, Turn.TURN),
    19: Movement(Direction.EAST, Turn.RIGHT),
    30: Movement(Direction.WEST, Turn.LEFT),
    31: Movement(Direction.WEST, Turn.LEFT),
    32: Movement(Direction.WEST, Turn.STRAIGHT),
    33: Movement(Direction.WEST, Turn.STRAIGHT)
}

# 只有西进口改变
PEAK_LANE_MOVEMENT_MAPPING3 = {
    16: Movement(Direction.EAST, Turn.STRAIGHT),
    17: Movement(Direction.EAST, Turn.STRAIGHT),
    18: Movement(Direction.EAST, Turn.STRAIGHT),
    19: Movement(Direction.EAST, Turn.RIGHT_TURN),
    30: Movement(Direction.WEST, Turn.LEFT),
    31: Movement(Direction.WEST, Turn.LEFT),
    32: Movement(Direction.WEST, Turn.STRAIGHT),
    33: Movement(Direction.WEST, Turn.STRAIGHT)
}

SAT_THRESHOLD = {
    Turn.LEFT: 0.85,
    Turn.STRAIGHT: 0.75,
    Turn.RIGHT: 0.9,
    Turn.TURN: 0.75
}

east_basic_lane_allocation = {Turn.STRAIGHT: 2, Turn.TURN: 0, Turn.RIGHT: 0}
east_flexible_lane_allocation = [
    {
        Turn.STRAIGHT: 1,
        Turn.TURN: 0.5,
        Turn.RIGHT: 0.5
    },
    {
        Turn.STRAIGHT: 0,
        Turn.TURN: 1,
        Turn.RIGHT: 1
    }
]
east_lane_allocation = LaneAllocation(east_basic_lane_allocation, east_flexible_lane_allocation)

west_basic_lane_allocation = {Turn.STRAIGHT: 2, Turn.LEFT: 1}
west_flexible_lane_allocation = [
    {
        Turn.STRAIGHT: 1,
        Turn.LEFT: 0,
    },
    {
        Turn.STRAIGHT: 0,
        Turn.LEFT: 1
    }
]
west_lane_allocation = LaneAllocation(west_basic_lane_allocation, west_flexible_lane_allocation)

cycle_length = 200
east_green_spilt = {Turn.STRAIGHT: 88 / cycle_length, Turn.TURN: 58 / cycle_length,
                    Turn.RIGHT: (1., 1 - (1 - 58 / cycle_length) * 0.3)}
west_green_spilt = {Turn.STRAIGHT: 88 / cycle_length, Turn.LEFT: 58 / cycle_length}


def east_vms_state_output_func(turn: Turn):
    vms_3 = {'vmsId': 3, 'laneId': 19, 'direction': 2}  # 最右侧
    vms_4 = {'vmsId': 4, 'laneId': 18, 'direction': 2}  # 次右侧

    if turn is Turn.STRAIGHT:
        vms_3['movement'] = 8
        vms_4['movement'] = 1
    elif turn is Turn.RIGHT:
        vms_3['movement'] = 3
        vms_4['movement'] = 4
    else:
        raise NotImplementedError('invalid priority turn for east entry link')
    return [vms_3, vms_4]


def west_vms_state_output_func(turn: Turn):
    vms_1 = {'vmsId': 1, 'laneId': 31, 'direction': 4}  # 下游
    vms_2 = {'vmsId': 2, 'laneId': 31, 'direction': 4}  # 上游

    if turn is Turn.STRAIGHT:
        vms_1['movement'] = 1
        vms_2['movement'] = 1
    elif turn is Turn.LEFT:
        vms_1['movement'] = 2
        vms_2['movement'] = 2
    else:
        raise NotImplementedError('invalid priority turn for west entry link')
    return [vms_1, vms_2]


east_vms = VMS(Turn.STRAIGHT, Turn.STRAIGHT, Turn.RIGHT, entity_state_func=east_vms_state_output_func)
# east_vms1 = VMS(Turn.STRAIGHT, Turn.STRAIGHT, Turn.TURN)
# east_vms2 = VMS(Turn.RIGHT_TURN, Turn.RIGHT_TURN, Turn.RIGHT)
west_vms = VMS(Turn.STRAIGHT, Turn.STRAIGHT, Turn.LEFT, entity_state_func=west_vms_state_output_func)

TRUCK_RATE = 0.2
PASSENGER_RATE = 1 - TRUCK_RATE
VEH_TYPE_FACTOR = (PASSENGER_RATE + 0.5 * TRUCK_RATE)
modified_straight_rate = STRAIGHT_SAT_RATE * VEH_TYPE_FACTOR
modified_left_rate = LEFT_SAT_RATE * VEH_TYPE_FACTOR
modified_turn_rate = TURN_SAT_RATE * VEH_TYPE_FACTOR
modified_right_rate = RIGHT_SAT_RATE * VEH_TYPE_FACTOR
east_demand = [TurnDemand(Turn.STRAIGHT, modified_straight_rate), TurnDemand(Turn.TURN, modified_turn_rate),
                TurnDemand(Turn.RIGHT, modified_right_rate)]
west_demand = [TurnDemand(Turn.STRAIGHT, modified_straight_rate), TurnDemand(Turn.LEFT, modified_left_rate)]

v_lane_east = VarianceLane(Direction.EAST, east_vms, SAT_THRESHOLD, east_demand, east_lane_allocation,
                            east_green_spilt)
v_lane_west = VarianceLane(Direction.WEST, west_vms, SAT_THRESHOLD, west_demand, west_lane_allocation,
                            west_green_spilt)

# controller = StaticIntersectionController(variance_lanes=[v_lane_east, v_lane_west],
#                                           lane_movement_mapping=LANE_MOVEMENT_MAPPING,
#                                           update_interval_sec=1800,
#                                           peak_lane_movement_mapping=PEAK_LANE_MOVEMENT_MAPPING,
#                                           peak_hour_range=(17, 20))
#
# tf_data_path = 'data/TrafficFlow_Logs3.log'
# for stat in read_file(tf_data_path):
#     controller.update_from_traffic_flow(stat[0])

peak1_pd = PlanDuration(PEAK_LANE_MOVEMENT_MAPPING3, 8, 10)
peak2_pd = PlanDuration(PEAK_LANE_MOVEMENT_MAPPING3, 16, 16.5)
peak3_pd = PlanDuration(PEAK_LANE_MOVEMENT_MAPPING2, 16.5, 19)
peak4_pd = PlanDuration(PEAK_LANE_MOVEMENT_MAPPING1, 19, 20)

normal1_pd = PlanDuration(LANE_MOVEMENT_MAPPING, 0, 8)
normal2_pd = PlanDuration(LANE_MOVEMENT_MAPPING, 10, 16)
normal3_pd = PlanDuration(LANE_MOVEMENT_MAPPING, 20, 24)
day_lane_plan = DayLanePlan([peak1_pd, peak2_pd, peak3_pd, peak4_pd, normal1_pd, normal2_pd, normal3_pd])

controller = StaticIntersectionController(variance_lanes=[v_lane_east, v_lane_west],
                                          lane_movement_mapping=LANE_MOVEMENT_MAPPING,
                                          update_interval_sec=1200,  # 1800
                                          history_lane_movement_mapping=day_lane_plan)


工作日方案

In [5]:
for period, lane_flow in weekday_flow.items():
    hour = period * (UPDATE_INTERVAL / 60)
    print(hour)
    controller.update_from_complete_data(lane_flow, hour)

0.0
Direction.EAST 0.08091329966329967 0.11407595651560072 -0.03316265685230105
Direction.WEST 0.09638047138047137 0.23813734158561747 -0.1417568702051461
0.3333333333333333
Direction.EAST 0.06828703703703705 0.11802908372158688 -0.04974204668454983
Direction.WEST 0.09017255892255892 0.260536398467433 -0.17036383954487405
0.6666666666666666
Direction.EAST 0.07849326599326599 0.11605252011859381 -0.03755925412532782
Direction.WEST 0.07817760942760943 0.25346301208370176 -0.17528540265609233
1.0
Direction.EAST 0.06660353535353535 0.11209939291260765 -0.0454958575590723
Direction.WEST 0.06502525252525253 0.18744473916887713 -0.1224194866436246
1.3333333333333333
Direction.EAST 0.05828373015873017 0.0928783203243178 -0.03459459016558763
Direction.WEST 0.05523989898989899 0.1616774030567134 -0.10643750406681442
1.6666666666666665
Direction.EAST 0.044928451178451186 0.07821544543272624 -0.03328699425427505
Direction.WEST 0.0497334455667789 0.13635917084192947 -0.08662572527515057
2.0
Directi

周末方案

In [7]:
controller = StaticIntersectionController(variance_lanes=[v_lane_east, v_lane_west],
                                          lane_movement_mapping=LANE_MOVEMENT_MAPPING,
                                          update_interval_sec=1200,  # 1800
                                          history_lane_movement_mapping=day_lane_plan)
for period, lane_flow in weekend_flow.items():
    hour = period * (UPDATE_INTERVAL / 60)
    print(hour)
    controller.update_from_complete_data(lane_flow, hour)

0.0
Direction.EAST 0.15151515151515152 0.05305555555555555 0.09845959595959597
Direction.WEST 0.16927083333333331 0.10978485116416152 0.059485982169171794
0.3333333333333333
Direction.EAST 0.12428977272727272 0.041388888888888885 0.08290088383838384
Direction.WEST 0.1294191919191919 0.12378426171529622 0.005634930203895688
0.6666666666666666
Direction.EAST 0.14086174242424243 0.054722222222222214 0.08613952020202022
Direction.WEST 0.12350063131313131 0.10610079575596819 0.017399835557163124
1.0
Direction.EAST 0.1116635101010101 0.05944444444444444 0.05221906565656566
Direction.WEST 0.11363636363636363 0.11125847332743886 0.0023778903089247683
1.3333333333333333
Direction.EAST 0.09232954545454546 0.049444444444444444 0.04288510101010101
Direction.WEST 0.09706439393939394 0.09357500736811084 0.0034893865712831013
1.6666666666666665
Direction.EAST 0.09114583333333333 0.04027777777777778 0.05086805555555555
Direction.WEST 0.08917297979797979 0.08325965222516948 0.005913327572810309
2.0
Dir